# Constructing an Event Dataset from CSS3.0 database tables 
## Prof. Gary L. Pavlis, Dept. Earth and Atmos. Sci., Indiana University


# 1. Overview
The purpose of this tutorial is to show how to use MsPASS to assemble a working data set using catalog data from an Antelope (CSS3.0) database as a template.  This is one example of many workflows that seismologists may want to use to assemble a working data set for processing by MsPASS.   We use it as an example because (a) it was a data set the author wanted to assemble, and (b) it is a useful example that exercises many of the tools seismologists would need to use to assemble a data set, and (c) it illustrates some of the idiosyncracies of real data downloaded from IRIS and the FDSN.  

The data set we will be building in this exercise are waveform segments appropriate for estimation of so called "receiver functions" from teleseismic P wave data.  We will be exploiting a useful but underutilized data set produced by Earthscope's USArray facility of analyst picks of teleseismic P waves recorded by USArray.   The concept I'm using here is that if the analyst can observe the phase well enough that he or she are confident enough to measure the P wave arrival time it is a candidate for receiver function processing.   We can thus use the picks as a screen to reduce the amount of debris from data with a signal so small that there is no way it can work for the processing we aim to accomplish.   An alternative workflow for accomplishing that prefiltering task is to use an automated procedure to screen junk later in the processing chain.  That would be feasible in MsPASS, but is not the approach used in this tutorial. 

The data set we will be using in this exercise was assembled with two tools the reader may already be familiar with:  (1) obspy's FDSN web service functions, and (2) Antelope's database tools.   As we use them I will explain how these data were assembled for your education, but the idea of this tutorial is that most of the data you will need for this exercise is already assembled for you to load.  The key educational objective is to help you understand how MongoDB can be used to manage raw data and what it takes to define what we call a working input data set for MsPASS.  

# 2. Creating An Empty MongoDB Database
We first need to create an instance of a MongoDB database into which we will be writing the data for this exercise.  For this tutorial I need to assume two things:
1.  You have a local instance of MongoDB running and accessible from the machine on which you are running this tutorial.  If that is not true, you need to follow the procedure described in this page_:<https://github.com/wangyinz/mspass/wiki/Using-MsPASS-with-Docker> for docker or this page: <https://github.com/wangyinz/mspass/wiki/Using-MsPASS-with-Singularity-(on-HPC)> for singularity.
2.  We assume the database we will be used, which we will call *raw_data_tutorial*, does not exist.   If you reenter this tutorial or something goes wrong and the tutorial database gets clobbered, we show below how to clear the contents quickly.\

With that (long winded) background, if you satisfy the assumptions above run the following:

In [1]:
from pymongo import MongoClient
from mspasspy.db import Database
client=MongoClient()
db=Database(client,'raw_data_tutorial')

The variable *db* is now a top level handle to MongoDB.   It is an abstraction that should be viewed as a handle to the "database" with the name "raw_data_tutorial".  Under this top level handle are "collections", which are MongoDB's equivalent to a *relation* (or equivalently a *table*) in a relational database.   You can see what collections are defined running the following at any time:

In [2]:
colnames=db.list_collections()
for nm in colnames:
    print(nm)

{'name': 'site', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('25fdf2ad-0b34-4775-8264-aa155da86823')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'raw_data_tutorial.site'}}
{'name': 'channel', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('756d4ad3-0fab-45b3-95ba-9830fcb52baa')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'raw_data_tutorial.channel'}}
{'name': 'source', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('7b30ffd6-20ad-4a44-a177-9e4b83e9438e')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'raw_data_tutorial.source'}}
{'name': 'arrival', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('9ee5e6a3-e3e1-4d9e-9ccd-3a8930871fe9')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'raw_data_tutorial.arrival'}}
{'name': 'seed_data.ensemble', 'type': 'collection', 'options': {}, 'info': {'readOn

For the present you should get nothing if you run that cell.  Later after we start adding data you can rerun that cell and see as we add a set of data to multiple "collections".  If you get output that is not clearly an error you can clear the database completely with this (very dangerous) command:

In [3]:
client.drop_database('raw_data_tutorial')
client=MongoClient()
db=client['raw_data_tutorial']

where we have repeated initalizing db as above.   

# 3. Retrieving raw data with web services
Over the past three decades IRIS developed a long suite of methods to retieve seismic data from their archives.  Many are still available, and without doubt the best alternative to the methods describe here is their "breqfast" request mechanism.   For the past decade, however, IRIS DMC has moved toward a solution using modern web services as their recommended data retrieval method.   We used obspy's FDSN mass download function to accomplish this objective.  (see <https://docs.obspy.org/packages/obspy.clients.fdsn.html> for a starting point on their documentation of this methodology.)  

You are welcome to use our actual procedure to download the data in this tutorial by running the python functions supplied with this tutorial.  From the directory you launched this tutorial cd to the directory *download_scripts* and run `python download2012.py`.   If you choose that route, however, you will need to be patient because when I ran this on a completely unloaded machine on a gigabit network it took about 5 days to complete.   Because that step is so time consuming we supply all the required file that process would yield in the directory where you are running this tutorial.   We do, however, only supply a small fraction of the actual waveform data as the size of that request is the main reason this takes 5 days - when I ran this script it returned approximately 0.65 TB.  

Since I doubt many readers will want to wait 5 days and consume over half a terrabyte of disk to run this tutorial we supply the following that we extracted from the output of that procedure:
1. We supply miniseed format waveform data for the first 10 events the download2012.py procedure will retieve.   You could recreate this in far shorter time if you hacked the script to count to 10 or just killed it after a few hours, but I recommend you start by looking in the directory wf/2012.   There you should see a set of "event" files.   Those files are not small because they were produced by concantenation of the miniseed files obspy downloads.  I did that to reduce the (absurd) number of files the original obspy script creates.  That is, obspy's function writes one file per channel per event.   Since the query I used in this script (all broadband stations running in 2012 within a rectangle around the lower 48 states of the US) yields over 1700 stations and around 5000 channels that gets out of control very quickly.  That means the data for that one year alone contains around 5 million files.  That is bad for a whole lot of reasons from simple things like many unix commands won't function on file lists for a directory that large to truly terrible performance problems on HPC systems (e.g. I found using the unix du query to query the data volume of the directory chain took hours to complete on a HPC disk farm using the Lustre file system at Indiana.)   I note we can use the concantenated miniseed files without any issues because of the way miniseed files are constructed and how they interact with obspy's reader we will be using later.  
2.  The procedure we will be using is driven by events obspy downloads as the new xml format recently defined for web services by the FDSN.  The file downloaded is `./eventdata/events_2012.xml`.  Note we will load that data, but override it later using an alternative event catalog.  I include that for completeness and hope it will not prove too confusing.  It is useful to emphasize the lesson that in passive data handling we often have to face the issue of multiple source parameters for the same seismic "event".   In this exercise we will have conflicting source information from two sources:  (a) the FDSN download that process this xml file, and (b) the catalog Array Network Facility analysts used to construct their event catalog.   
3.  During downloading obypy dynamically assembles station metadata in a custom scheme used by the download procedure using in the download2012.py script.  Some of the details of how they (obspy) handle this can be found in their documentation, but a key point for this tutorial is you will not be able to reproduce the station metadata we supply here exactly unless you run the download2012.py script to completion. We supply the output in the directory site_2012.dir, which is the name their script creates and populates with the files you will find there.  That directory contains 1712 small files.  There is one file for each seed network-station code combinations of data downloaded.  Each file is a recently standardized FDSN web service xml representation of station metadata.  Seismologists unfamiliar with station xml should realize it is simply a different format to contain the same information previously stored in the SEED blockettes used to define station metadata.  That information is also sometimes stored as "dataless seed" files (e.g. the Array Network Facility distributes these through their web site at <https://anf.ucsd.edu/data/products/dataless_sta/>).   

The next three sections describe how we assemble complete metadata for the tutorial data set we will build and load into MongoDB here.  The sections are headed by a useful classification of the type of data involved:  (a) receiver metadata, (b) source metadata, and (c) waveform data and metadata. 

# 4. Assembling the pieces
## 4.1. Receiver metadata
With tools we presently supply this is by far the easiest thing to do.   For this tutorial, just run this simple set of command:

In [8]:
from obspy import read_inventory
inv=read_inventory('site_2012.dir/*.xml')
db.save_inventory(inv)

BK : BRIB : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   37.91886 -122.15179 0.21969999999999998
loc code section coordinates:   37.91881 -122.15176 0.2189
BK : HAST : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   36.3887 -121.5514 0.542
loc code section coordinates:   36.3887 -121.5514 0.5395
BK : HATC : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   40.8161 -121.46117 1.0092999999999999
loc code section coordinates:   40.8161 -121.46117 1.0086
BK : HELL : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   36.68011 -119.02282 

US : ECSD : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   43.7337 -96.6141 0.478
loc code section coordinates:   43.7337 -96.6141 0.43289999999999995
US : GOGA : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   33.4112 -83.4666 0.1165
loc code section coordinates:   33.4112 -83.4666 0.083
US : KSU1 : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   39.1009 -96.6094 0.317
loc code section coordinates:   39.1009 -96.6094 0.2963
UU : SRU :   (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   39.11083 -110.52383 1.804
loc code section coordi

(1712, 5148, 1712, 5148)

You will notice this generates a fair number of warnings.  Those warning are intentional and should be inspected if you repeat this process for other data.   In all cases here these seem to be a combination of two issues: (1) many stations seem to have a mismatch in precision of location information stored in the station and channels section of the xml data, and (2) some of the mismatches in the station and channels section reflect a real geometric difference (e.g. RSSD loc code 00 is a borehole instrument so the elevation field differs from the station location which is at surface elevation).  

Notice for this process we are using obspy's read_inventory function to crack the station xml files and turn them into something we can manipulate.  That something is their Inventory object (<https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.Inventory.html#obspy.core.inventory.inventory.Inventory>) which is pretty much a python translation of the station xml.  The save_inventory method of our Database class translates the Inventory data into the structure MongoDB requires to manage the data we just saved.   In MongoDB lingo that information is a "document".  We write one document in a collection called *site* for each "station" defined by a unique SEED network (net) and station (sta) code.  Each station always has one or more "channels" associated with it.  In SEED each channel is defined by two additional keys:  a channel (chan) code, and a location (loc) code.  SEED thus indexes a single channel of seismic data with a key defined by all four code:  net, sta, chan, and loc.  In MsPASS we map the Channels data assembled in the obspy Inventory object to produce one or more MongoDB documents we write to a collection we call *channel*. 

A further huge complication is that station and channel metadata can and often are time variable.   The most common example is a sensor change where the response data and/or the orientation of the instrument can change.  Furthermore, all channels operate only over a finite time range.  Hence, the *channel* collection commonly contain multiple documents for some channels.   There are no examples in this tutorial data with a *site* having multiple documents for the same SEED net-sta keys, but the schema we use allows that by using the time interval defined by attributes *starttime* and *endtime* as an additional constraint.   (Note using net-sta-opeational_time as a unique site idenifier was borrowed completely from CSS3.0 which uses that same concept.)  

To clarify this further, you can run the following optional code block to get a basic report of the number of channels per site.  You can also easily alter the report to print station location information using the interactive feature of jupyter.

In [9]:
def print_station_report(db):
    """
    Simple example function to print a basic report on contents of the site and channel collections.
    The example uses the low level MongoDB API using json.  
    :param db:  is a mspasspy.db Database object assumed pointed at an operating MongoDB server
    """
    dbsite=db.site
    dbchannel=db.channel
    curs=dbsite.find({})
    print('net sta Number_channels')
    for doc in curs:
        net=doc['net']
        sta=doc['sta']
        query=dict()
        query['net']=net
        query['sta']=sta
        n=dbchannel.count_documents(query)
        print(net,sta,n)

print_station_report(db)
        

net sta Number_channels
2G IUGFS 3
7D FS01B 3
7D FS05B 3
7D FS06B 3
7D FS09B 3
7D FS14B 3
7D G02B 3
7D G03A 3
7D G03B 3
7D G04B 3
7D G05B 3
7D G10B 3
7D G11B 3
7D G12B 3
7D G13B 3
7D G19B 3
7D G20B 3
7D G21B 3
7D G22B 3
7D G27B 3
7D G28B 3
7D G29B 3
7D G30A 3
7D G30B 3
7D G35B 3
7D G36B2 3
7D G37B 3
7D J06A 3
7D J06B 3
7D J09B 3
7D J10B 3
7D J11B 3
7D J18B 3
7D J19B 3
7D J20B 3
7D J23A 3
7D J23B 3
7D J25A 3
7D J27B 3
7D J28A 3
7D J28B 3
7D J29A 3
7D J30A 3
7D J31A 3
7D J33A 3
7D J35A 3
7D J36A 3
7D J37A 3
7D J38A 3
7D J39A 3
7D J43A 3
7D J44A 3
7D J45A 3
7D J46A 3
7D J47A 3
7D J48A 3
7D J48B 3
7D J52A 3
7D J53A 3
7D J54A 3
7D J55A 3
7D J57A 3
7D J61A 3
7D J63A 3
7D J63B 3
7D J65A 3
7D J67A 3
7D J68A 3
7D J73A 3
7D M01A 3
7D M02A 3
7D M07A 3
7D M08A 3
7D M11B 3
7D M12B 3
7D M14B 3
AE 113A 3
AE 319A 3
AE U15A 3
AE W13A 3
AE X16A 3
AE X18A 3
AE Y14A 3
AZ BZN 3
AZ CPE 3
AZ CRY 3
AZ FRD 3
AZ HWB 3
AZ KNW 3
AZ LVA2 3
AZ MONP2 3
AZ PFO 3
AZ RDM 3
AZ SCI2 3
AZ SMER 3
AZ SND 3
AZ SOL 3
AZ TRO 3

TA P47A 3
TA P48A 3
TA P49A 3
TA P50A 3
TA P51A 3
TA P52A 3
TA P53A 3
TA P55A 3
TA Q24A 3
TA Q34A 3
TA Q35A 3
TA Q36A 3
TA Q37A 3
TA Q38A 3
TA Q39A 3
TA Q40A 3
TA Q41A 3
TA Q42A 3
TA Q43A 3
TA Q44A 3
TA Q45A 3
TA Q46A 3
TA Q47A 3
TA Q48A 3
TA Q49A 3
TA Q50A 6
TA Q51A 3
TA Q52A 3
TA Q55A 3
TA R11A 3
TA R34A 3
TA R35A 3
TA R36A 3
TA R37A 3
TA R38A 3
TA R39A 3
TA R40A 3
TA R41A 3
TA R42A 3
TA R43A 3
TA R44A 3
TA R45A 3
TA R46A 3
TA R47A 3
TA R48A 3
TA R49A 3
TA R50A 3
TA R51A 3
TA R52A 3
TA R58B 3
TA S22A 3
TA S34A 3
TA S35A 3
TA S36A 3
TA S37A 3
TA S38A 3
TA S39A 3
TA S40A 3
TA S41A 3
TA S42A 3
TA S43A 3
TA S44A 3
TA S45A 3
TA S46A 3
TA S47A 3
TA S48A 3
TA S49A 3
TA S50A 3
TA S51A 3
TA S52A 3
TA SFIN 3
TA SPMN 3
TA SUSD 3
TA T25A 3
TA T34A 3
TA T35A 3
TA T36A 3
TA T37A 3
TA T38A 3
TA T39A 3
TA T40A 3
TA T41A 3
TA T42A 3
TA T43A 3
TA T44A 3
TA T45A 3
TA T46A 3
TA T47A 3
TA T48A 3
TA T49A 3
TA T50A 3
TA T51A 3
TA T52A 3
TA TASL 3
TA TASM 3
TA TFRD 3
TA TIGA 3
TA TPFO 3
TA TUL1 3
TA U32A 3


This illustrates most of the stations have 3 channels of data per site.  A couple of good exercises here for the beginner are:
1.  Alter the function to also print site coordinates.
2.  Print only stations where the count is not exactly 3.
3.  Alter the script to also print a list of channel codes (key chan) and location codes (key loc).  Hint: this one is much harder because it requires replacing the count_documents call with a find operator followed by a second loop.
  
A final point about the Database object in MsPASS and the save_inventory method.  That method can be called repeatedly with duplicate data and the function will assure pure duplicates are not added to the site or channel collections. That makes adding collections of stationxml files downloaded independently easy to add without worries about adding clutter.  That is common, for example, when downloading data like the TA by year; each download run will produce a new pile of stationxml files with many duplicates. To demonstrate this, run this block again which will attempt to save the same Inventory object read earlier a second time. 

In [21]:
db.save_inventory(inv)

BK : BRIB : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   37.91886 -122.15179 0.21969999999999998
loc code section coordinates:   37.91881 -122.15176 0.2189
BK : HAST : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   36.3887 -121.5514 0.542
loc code section coordinates:   36.3887 -121.5514 0.5395
BK : HATC : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   40.8161 -121.46117 1.0092999999999999
loc code section coordinates:   40.8161 -121.46117 1.0086
BK : HELL : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   36.68011 -119.02282 

NE : WVL :   (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   44.5646 -69.6617 0.05
loc code section coordinates:   44.564602 -69.661697 0.05
US : ECSD : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   43.7337 -96.6141 0.478
loc code section coordinates:   43.7337 -96.6141 0.43289999999999995
US : GOGA : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   33.4112 -83.4666 0.1165
loc code section coordinates:   33.4112 -83.4666 0.083
US : KSU1 : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   39.1009 -96.6094 0.317
loc code section coordina

(0, 0, 1712, 5148)

To see the key point you will need to scroll to the bottom of the output to bypass the warning messages.  The summary should say that 0 site and 0 channel records were saved.  

## 4.2. Source metadata 
### 4.2.1. Extracting data from stationml files
MsPASS has a similar, simple method to import event (source) data stored as a file in the so called QuakeML format defined by the FDSN (<https://www.fdsn.org/webservices/>) for web services.  Obspy has a direct translation of the XML format file to a python object they call a Catalog (<https://docs.obspy.org/packages/autogen/obspy.core.event.Catalog.html#obspy.core.event.Catalog>) object.  You can load the QuakeML data file for this tutorial by a pair of commands very similar to that used above to read the station metadata:

In [10]:
from obspy import read_events
#db.drop_collection('source')
cat=read_events('eventdata/events_2012.xml',format='QUAKEML')
n=db.save_catalog(cat)
print('Number of events stored by save_catalog in source collection=',n)

Number of events stored by save_catalog in source collection= 961


Note the method returns the count of the number of documents added that is printed with an explanation in the last line.  In MsPASS we store event information in a separate collection we call *source*.

It is important to realize one key difference between this method and save_inventory.   Unlike seismic instruments that fixed objects an earthquake location is intrinsically fuzzy.  Multiple institutions often estimate the location from a different mix of instruments.   The CSS3.0 schema has a fairly elaborate way to handle this problem that causes major bookkeeping headaches for all seismic network operators or any experiment that requires preparing an event bulletin (meaning location estimates and a set of measure arrival times).   In MsPASS we treat that problem as a preprocessing issue that we will need to address below.   That is, in the next section we will add a second set of earthquake location estimates to our *source* collection and use a method to define which document in *source* to link to other documents.  A WARNING, however, is you should not rerun save_catalog on data from the same input file.  It will silently insert exact duplicates of the first run. 

### 4.2.2. Import data from Antelope (CSS3.0 database)
#### 4.2.2.1. Background
Antelope is a software package developed and supported by a small company called Boulder Real Time Technologies (BRTT).   A component of Antelope is a specialized relational database package sometimes called Datascope because the current version is a descendent of an open source package with that name developed in the 1990s.   There are several key points about Antelope that are important to recognize as a preface to this tutorial:
- Although Antelope is a commerical package it is widely available and used in the US because of a generous licensing arrangement by BRTT.  Thus most readers of this tutorial in the US can obtain Antelope at no cost.  If that is you then you can recreate the steps here and create the catalog file we will be using yourself.  For others we supply an export file that can be used directly.  If you are the later, you need only skim this section to get an idea of what the file is and how it was generated. 
- Antelope is used by a number of regional seismic networks, notably the Alaska and Nevada networks.   More importantly to many seismologists is the fact that Antelope was the processing system used to manage the real time data from the USArray project at the Array Network Facility.   Important data relevant to this tutorial is the set of monthly catalogs of data distributed by ANF at this location:  <https://anf.ucsd.edu/tools/events/>.  In this tutorial we will be using data from the USArray recorded in 2012.   The unpacked tar files are found in a directory *anf* in the folder where you launched this tutorial.   I used a shell script found in a different directory, *catalog_data*, to use Antelope database command line tools to create a file *./catalog_data/usarray_tele2012.txt*. That script is found in *./catalog_data/export2012.csh* and for reference is this:
```
!/bin/csh
set yr=2012
set outfile="usarray_tele${yr}.txt"
#rm $outfile
set dbdir="../anf"
foreach mon (01 02 03 04 05 06 07 08 09 10 11 12)
  set dbname=usarray_${yr}_${mon}
  set dbpath=${dbdir}/events_${dbname}/$dbname
  echo Working on database $dbname
  dbjoin ${dbpath}.event origin \
  | dbsubset - "orid==prefor" \
  | dbjoin - netmag \
  | dbsubset - "magnitude>5.0" \
  | dbjoin - assoc arrival \
  | dbsubset - "assoc.delta>30.0 && assoc.delta<95.0" \
  | dbselect - event.evid origin.lat origin.lon origin.depth origin.time origin.mb origin.ms assoc.sta assoc.phase arrival.iphase assoc.delta assoc.seaz assoc.esaz assoc.timeres arrival.time arrival.deltim \
  >> $outfile
```
- Although that shell script is not elegant it accomplishes the object I was seeking.  It produces the main attributes I need from the css3.0 database for defining the data set we will build in this tutorial.   That is, a subset of teleseismic events recorded by USArray in 2012.   In particular, our objective is to produce a data set appropriate for P wave receiver function analysis.   We select only larger events (the *magnitude>5.0* dbsubset command) and arrivals measured at stations between 30 and 95 degrees epicentral distances (the dbsubset querry using *assoc.delta*).  We use Antelope's dbselect command to extract only attributes we might need for this analysis. You should recognize the argument list to dbselect does not extract all attributes possible, but the specific list there is intimately linked to a step below when we parse the output of this script (the file *./catalog_data/usarray_tele2012.txt*).  
- The approach we use here can and should be thought of as a way to general way export data in an antelope css3.0 database to MsPASS.  Here the emphasis is "catalog" data, but the same general idea could be done for any relational database view that can be constructed with Antelope tools.   This particular example is a custom export to match python functions we will be used that are distributed with MsPASS and which we will be using below.

#### 4.2.2.2. The *arrival* collection
Previously we added data to collections called *site*, *channel*, and *source*.   Those three collections contain fundamental data comparable to geometry in seismic reflection processing and are required information for most processing work flows with MsPASS.   For raw processing, however, we need an additional concept that is not as univeral as those three core concepts.   That new concept is what seismologists universally call an "arrival".  We define an *arrival document* as a set of parametric data measured in a time window of data that defines a particular "seismic phase".  For this tutorial we will be working with arrivals defined by teleseismic P waves.  The main parametric data of interest for this tutorial will be the measured arrival time of P by an analyst at the Array Network Facility.   In the script above, that is the CSS3.0 attribute with the tag "arrival.time".  

We first want to load the output from the shell script described in the previous section.   Note compared to what we did previously, this data set is huge.  The table that script generates has 353,482 rows.  Run the following script to load that data in this tutorial:

In [3]:
from mspasspy.preprocessing.css30.dbarrival import load_css30_arrivals
valret=load_css30_arrivals(db,'catalog_data/usarray_tele2012.txt')

We can confirm this worked and inserted the number we expected with this pair of commands.

In [4]:
dbarrival=db.arrival
print('Number of arrivals added=number of documents in arrival=',dbarrival.count_documents({}))


Number of arrivals added=number of documents in arrival= 353482


#### 4.2.2.3. CSS3.0 Limitation History and Solution - background you may elect to skip
The CSS3.0 schema was developed in the late 1970s when storage and computer speeds were orders of magnitude smaller than today.  At the time few had the vision to think the number of seismic instruments on the planet would grow as large as it has.  The schema relied on the concept of an alphanumeric "station code", which had a long history in seismology as a filing system for analog records.  Hence, it was natural to use station codes as an index for many database tables.  One of them was the arrival table, which is what we are working to translate here. The catalog tables (event, origin, assoc, and arrival) use the station code as a primary join key.  The data we just loaded extracted that field in the shell script above a assoc.sta, which is the copy of the station code in the CSS3.0 assoc table.

In the mid 1980s with the birth of the Global Seismic Network and IRIS the global seismic community developed the Standard Earthquake Exchange Data (SEED) format.  The committee that drafted the standard recognized that a station code alone created a problem by limiting the namespace to define a unique key to identify a single channel of seismic data.  They elected to define a four part key that has constrained earthquake data management since that decision was made.  That four part key in MsPASS have the abbreviations:  net, sta, chan, and loc.  (Obspy refers to these in the more verbose form network, station, channel, and location.)  The SEED standard created a collision with the CSS3.0 schema standard because CSS3.0 did not use the concept of a network or location code.   The developers of the original Datascope package, the ancestor of Antelope's database, had a significant code base that utilized the CSS3.0 schema - notably dbpick written by Danny Harvey that remains a workhorse in Antelope. Hence, Datascope did not use a revised schema but instead built their tools directly on the CSS3.0 schema.  As their code base grew it became too difficult for them the move to a different schema that would handle SEED data more easily.  They instead chose a band-aid solution that had several elements:
1.  The station code tag was treated as a unique identifier in all tables as defined in CSS3.0
2.  Similarly the channel (chan) code was treated as a unique identifier for each seismic channel as defined in CSS3.0
3.  If a SEED (miniseed) file was read that created a name collision with the station code (i.e. two or more stations with the same station code but a different network code) the name of all secondary stations encountered would use an altered station code.  (e.g. in this data we have net=BK and sta=HELL and net=TA and sta=HELL.  If TA:HELL had been handled earlier BK:HELL would be tagged as HELLBK.  Similarly if BK:HELL were handled first TA:HELL would be treated as HELLTA.)   
4.  Many observatory style seismic stations have multiple sensors at the same location.  GSN stations, for example, all (at least nearly all) have 2 or more sensors.   Unfortunately, the SEED standard also adopted ridid rules about how a channel codes should be defined that further limited the namespace (<https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/>).  Hence, we have only a small namespace for channel names like BHZ, HHZ, BH1, HHE, etc.  A SEED defined channel with that restriction will always be ambiguous if a seismic station has more than one sensor.   e.g. Many GSN stations have a BHZ channel defined for location codes 00 and 01.  To adapt CSS3.0 to this ambiguity the authors of Datascope elected to use similar rules as for the station information.  The first channel code with a unique name would drop the loc code and subsequently appearances would include the loc code.  

### 4.3. Resolving arrival ambiguities
The point of the admittedly long winded discussion above about css3.0 limitations and net:sta:chan:loc code in SEED is that the arrival data we just loaded currently have an ambiguous relationship with the data we stored in the site and channel collections.  To see that run the cell below:

In [5]:
from mspasspy.preprocessing.css30.dbarrival import find_duplicate_sta
dups=find_duplicate_sta(db)
print('There are ',len(dups),' stations in site with ambiguous sta codes')
print('List of station with multiple net codes for that sta code')
for k in dups:
    print(k,dups[k])

There are  58  stations in site with ambiguous sta codes
List of station with multiple net codes for that sta code
J31A {'TA', '7D'}
J33A {'TA', '7D'}
J35A {'TA', '7D'}
J36A {'TA', '7D'}
J37A {'TA', '7D'}
J38A {'TA', '7D'}
J39A {'TA', '7D'}
J43A {'TA', '7D'}
J45A {'TA', '7D'}
J46A {'TA', '7D'}
J47A {'TA', '7D'}
J48A {'TA', '7D'}
J52A {'TA', '7D'}
J54A {'TA', '7D'}
J55A {'TA', '7D'}
PFO {'AZ', 'II'}
SCI2 {'AZ', 'CI'}
PER {'Y5', 'CI'}
WES {'CI', 'NE'}
CCRK {'UW', 'HW'}
DDRF {'UW', 'HW'}
PHIN {'UW', 'HW'}
U32A {'OK', 'TA'}
X37A {'OK', 'TA'}
A01 {'XD', 'YX'}
A02 {'XD', 'YX'}
A03 {'XD', 'YX'}
A04 {'XD', 'YX'}
A05 {'XD', 'YX'}
A06 {'XD', 'YX'}
A08 {'XD', 'YX'}
A09 {'XD', 'YX'}
A10 {'XD', 'YX'}
A11 {'XD', 'YX'}
A13 {'XD', 'YX'}
A14 {'XD', 'YX'}
A15 {'XD', 'YX'}
B01 {'XD', 'YX'}
B02 {'XD', 'YX'}
C02 {'XD', 'YX'}
D02 {'XT', 'Z9'}
D03 {'XT', 'Z9'}
D04 {'XT', 'Z9'}
D05 {'XT', 'Z9'}
D06 {'XT', 'Z9'}
D07 {'XT', 'Z9'}
D08 {'XT', 'Z9'}
D09 {'XT', 'Z9'}
D10 {'XT', 'Z9'}
D11 {'XT', 'Z9'}
D12 {'XT', 'Z9

The above shows we have 58 potentially ambiguous station codes.  That small function, whoever, just looks at site.  The more relevant output for the problem at hand, which is to unambiguously associate arrival times we loaded in arrival with the right documents in site and channel, comes form this related function:

In [6]:
from mspasspy.preprocessing.css30.dbarrival import check_for_ambiguous_sta
# this is using the dups list created above and extracting only the sta code into a python list
stalist=list()
for k in dups:
    stalist.append(k)
retval=check_for_ambiguous_sta(db,stalist)

station count
J31A 190
J33A 209
J35A 161
J36A 518
J37A 512
J38A 481
J39A 761
J43A 645
J45A 87
J46A 133
J47A 178
J48A 149
J52A 45
J54A 7
J55A 4
PFO 983
SCI2 449
PER 0
WES 0
CCRK 0
DDRF 0
PHIN 0
U32A 80
X37A 87
A01 0
A02 0
A03 0
A04 0
A05 0
A06 0
A08 0
A09 0
A10 0
A11 0
A13 0
A14 0
A15 0
B01 0
B02 0
C02 0
D02 0
D03 0
D04 0
D05 0
D06 0
D07 0
D08 0
D09 0
D10 0
D11 0
D12 0
D13 0
D14 0
D15 0
D17 0
D18 0
D20 0
D21 0


This shows we can ignore some stations like A01 that have no picks in arrival, but we have a big issue with stations like PFO with hundreds of picks in arrival.   

We can patch part of this problem using a nonstandard Datascope (Antelope) table I obtained directly from ANF, that is, unfortunately, not distributed from the ANF web site.  The reasons for that are complicated and irrelevant, but for now the key data is in the file snetsta/usarray.snetsta.  The snetsta table is a patch method Antelope uses to workaround the station ambiguity problem.  Here are few selected lines from that file that show how we use it:
```
TA       934A   934A    1503120361.98014
TA       834A   834A    1503120736.28809
TA       835A   835A    1503120736.92130
AE       X16A   X16AAE  1600860674.47829
AZ       BZN    BZN     1600860676.10732
AZ       CPE    CPE     1600860677.78964
```
In this table the net code is column 1, the seed station code is column2, and the sta code we can expect to find in arrival is column 3.  Column 4 is an internal time stamp used in Datascope that defines the epoch time of that tuple was last updated.  The key tuple to note is the 4th one in this list.  Note net AE and sta X16A are aliased to X16AAE.   The next step will exploit this to fix part of our ambiguity issue.

In [7]:
from mspasspy.preprocessing.css30.dbarrival import (parse_snetsta,set_netcode_snetsta)
staxref=parse_snetsta('snetsta/usarray.snetsta')
print('Example station cross reference created from snetsta for sta=120A:',staxref['120A'])
print('Using staxref to set net codes in arrival collection')
rettuple=set_netcode_snetsta(db,staxref)
print('Number of arrival documents scanned=',rettuple[0])
print('Number of arrival documents updated=',rettuple[1])

Example station cross reference created from snetsta for sta=120A: {'fsta': '120A', 'net': 'TA'}
Using staxref to set net codes in arrival collection
353482
Number of arrival documents scanned= 353482
Number of arrival documents updated= 98149


I intentionally didn't print all the components of the rettuple.   We will use the 3rd component of that tuple in the next step.  First, let's see what it contains:

In [8]:
stations_not_found=rettuple[2]
print('Number of stations with unset net code=',len(stations_not_found))
print(stations_not_found)

Number of stations with unset net code= 666
{'V46A', '347A', 'I42A', 'E40A', 'Q36A', 'U44A', 'K52A', 'CHGQ', 'Z44A', '246A', '552A', 'E31A', 'U45A', 'R45A', 'Y42A', 'U50A', 'TASL', 'ORHO', 'D04D', 'A04D', 'W46A', 'S34A', 'B35A', 'G39A', '444A', '441A', 'X43A', 'P43A', 'E33A', 'TCOL', 'N55A', 'LSQQ', 'F34A', 'J48A', 'D50A', 'U51A', 'F33A', 'O42A', 'Y53A', 'V53A', 'T39A', 'O43A', 'N38A', 'MCWV', 'E35A', '447A', 'POKR', 'F45A', '250A', '541A', '442A', 'W38A', 'L38A', '452A', '656A', '857A', 'W43A', 'X36A', 'U49A', 'H56A', 'T50A', '557A', 'O49A', 'Q48A', 'Q46A', 'H37A', 'Y36A', 'M49A', 'N47A', 'M42A', 'H41A', 'T51A', 'F43A', 'M41A', 'Z39A', 'Z36A', 'K51A', 'W37B', 'L42A', 'K49A', 'X45A', '355A', 'J34A', 'F46A', 'PEMO', 'H38A', 'Y48A', 'J41A', '356A', 'BW06', 'Q43A', 'F36A', 'X51A', '758A', 'JFWS', 'T44A', 'BASO', 'D41A', 'STCO', 'X38A', 'P37A', 'I52A', 'CBKS', 'T42A', 'S44A', 'Q47A', 'N40A', 'T43A', 'P38A', 'G46A', 'E43A', 'N46A', 'ERPA', 'P34A', 'O39A', 'W50A', '236A', 'W36A', 'U43A', 'V5

Which shows snetsta solved only part of our problem.   We have a devil of problem to fix here with 666 stations having no entry in the snetsta table we used. (Bad joke, but couldn't resist.) 

The first fix we will apply makes a simple assumption that is reasonable here and probably usually would be for other data sets assembled by a similar mechanism (i.e. web service download of waveforms from IRIS and the FDSN).  We assume that the station data we wrote in the site collection from the download procedure is close to definitive but not completely.  The exact assumption is that if there is a unique match for a station code in site set the net code for an arrival to that value.  If it is not found put it in one list to be returned.  If it is ambiguous, meaning there are multiple net codes for the same sta name, return it in a different list. The next code block applies that algorithm:

In [9]:
from mspasspy.preprocessing.css30.dbarrival import set_netcode_from_site
rt_setsite=set_netcode_from_site(db)
nprocessed=rt_setsite[0]
nupdated=rt_setsite[1]
ambiguous=rt_setsite[2]
sta_not_found=rt_setsite[3]
print('set_netcode_from_site processed ',nprocessed,' documents in arrival collection')
print('number of documents with net code update=',nupdated)

set_netcode_from_site processed  353482  documents in arrival collection
number of documents with net code update= 240699


You will likely find that that function takes a while to run because it is doing a lot of updates in a modest sized collection.   Let's print the two larger containers returned in the rt_setsite tuple in a pretty form:

In [10]:
from obspy import UTCDateTime
print('ambiguous site entries matched against arrival')
# first sort by sta - put tuples in a list container first to allow sorting
amblist=list()
for t in ambiguous:
    amblist.append(t)
amblist.sort(key = lambda x: x[1])
for t in amblist:
    print(t[0],t[1],UTCDateTime(t[2]),' to ',UTCDateTime(t[3]))
print('These sta codes had entries in arrival but were not found in site')
for nullsta in sta_not_found:
    print(nullsta)

ambiguous site entries matched against arrival
7D J31A 2011-11-21T00:00:00.000000Z  to  2012-05-16T23:59:59.000000Z
TA J31A 2010-07-31T00:00:00.000000Z  to  2012-05-18T23:59:59.000000Z
7D J33A 2011-10-16T00:00:00.000000Z  to  2012-07-19T23:59:59.000000Z
TA J33A 2010-07-27T00:00:00.000000Z  to  2012-05-17T23:59:59.000000Z
TA J35A 2010-07-29T00:00:00.000000Z  to  2012-05-16T23:59:59.000000Z
7D J35A 2011-10-19T00:00:00.000000Z  to  2012-07-18T23:59:59.000000Z
7D J36A 2011-10-19T00:00:00.000000Z  to  2012-07-18T23:59:59.000000Z
TA J36A 2010-10-07T00:00:00.000000Z  to  2012-09-16T23:59:59.000000Z
TA J37A 2010-10-06T00:00:00.000000Z  to  2012-09-10T23:59:59.000000Z
7D J37A 2011-11-28T00:00:00.000000Z  to  2012-05-16T23:59:59.000000Z
7D J38A 2011-11-23T00:00:00.000000Z  to  2012-05-17T23:59:59.000000Z
TA J38A 2010-10-11T00:00:00.000000Z  to  2012-09-14T23:59:59.000000Z
7D J39A 2011-11-22T00:00:00.000000Z  to  2012-05-17T23:59:59.000000Z
TA J39A 2011-06-12T00:00:00.000000Z  to  2013-06-03T23:5

So, we clearly need to do some detective work to sort this out.   

The ambiguous station list is easy to deal IF we make an assumption.   If you go to IRIS's Metadata aggregator page for query by network, <http://ds.iris.edu/SeismiQuery/by_network.html>, you will learn that 7D is a set of OBS instruments deployed in Cascadia in 2012.  The TA code is for the USArray Transportable Array.  The ANF received TA data in real time and the focus of their efforts was the data received in real time.  To the best of my knowledge they did not go back and pick the OBS data that was not accessible until well after 2012 - the year of all arrival picks we are trying to sort out here.  Hence, the assumption we make is that all of those picks can safely assign TA as the net code.   The only exception is X37A, but the timestamps suggest this is an example of a TA station adopted by the OK network.  Let's check the coordinates to verify that is true before proceeding:

In [11]:
dbsite=db.site
stdata=dbsite.find({'sta':'X37A'})
for doc in stdata:
    print(doc['net'],doc['sta'],doc['latitude'],doc['longitude'],doc['elevation'])

OK X37A 34.5892 -95.3713 0.207
TA X37A 34.589199 -95.3713 0.207


This shows they are the same location within roundoff error.

With that background we can now force all the stations in the ambiguous list to TA with the following code:

In [12]:
from mspasspy.preprocessing.css30.dbarrival import set_netcode_time_interval
# first use a set container to find all unique names
sta_to_force=set()
for t in ambiguous:
    sta_to_force.add(t[1])
for sta in sta_to_force:
    n=set_netcode_time_interval(db,sta,net='TA')
    print('set net code to TA for ',n,' arrival documents with picks for station=',sta)

set net code to TA for  645  arrival documents with picks for station= J43A
set net code to TA for  161  arrival documents with picks for station= J35A
set net code to TA for  149  arrival documents with picks for station= J48A
set net code to TA for  481  arrival documents with picks for station= J38A
set net code to TA for  178  arrival documents with picks for station= J47A
set net code to TA for  133  arrival documents with picks for station= J46A
set net code to TA for  87  arrival documents with picks for station= J45A
set net code to TA for  87  arrival documents with picks for station= X37A
set net code to TA for  518  arrival documents with picks for station= J36A
set net code to TA for  209  arrival documents with picks for station= J33A
set net code to TA for  45  arrival documents with picks for station= J52A
set net code to TA for  761  arrival documents with picks for station= J39A
set net code to TA for  512  arrival documents with picks for station= J37A
set net code to

Finally, we have to deal with the orphans, which are the stations held in the set container with the name sta_not_found.  We will do that with this little 

In [13]:
from obspy.clients.fdsn import Client
webclient=Client('IRIS')
starttime=UTCDateTime("2004-01-01T00:00:00.000")
endtime=UTCDateTime("2016-12-31T00:00:00.000")
write_dir="orphanxml"
for sta in sta_not_found:
    print('Attempting to download xml data for station=',sta)
    try:
        inv=webclient.get_stations(network="*",station=sta,location="*",
                            channel="BH*",
                            starttime=starttime,endtime=endtime,
                            level="response")
    except:
        print("No B channel data for station=",sta," for any net code")
        print("Ignoring this station")
        continue
    print('downloaded as this inventory object by obspy:')
    print(inv)
    fname=write_dir+'/'+sta+'.xml'
    print('writing as xml data to file=',fname)
    inv.write(fname,format='STATIONXML')
    print('saving to MongoDB site and channel collections')
    db.save_inventory(inv,verbose=True)

Attempting to download xml data for station= ELFO
No B channel data for station= ELFO  for any net code
Ignoring this station
Attempting to download xml data for station= MATQ
No B channel data for station= MATQ  for any net code
Ignoring this station
Attempting to download xml data for station= TYNO
No B channel data for station= TYNO  for any net code
Ignoring this station
Attempting to download xml data for station= LATQ
No B channel data for station= LATQ  for any net code
Ignoring this station
Attempting to download xml data for station= ACTO
No B channel data for station= ACTO  for any net code
Ignoring this station
Attempting to download xml data for station= CHGQ
No B channel data for station= CHGQ  for any net code
Ignoring this station
Attempting to download xml data for station= CLWO
No B channel data for station= CLWO  for any net code
Ignoring this station
Attempting to download xml data for station= POKR
downloaded as this inventory object by obspy:
Inventory created at 2

downloaded as this inventory object by obspy:
Inventory created at 2020-11-16T21:40:56.000000Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.47
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2004-01-01...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			TA
		Stations (1):
			TA.EPYK (Eagle Plains, Yukon Territory, CAN)
		Channels (6):
			TA.EPYK..BHZ, TA.EPYK..BHN, TA.EPYK..BHE, TA.EPYK.01.BHZ, 
			TA.EPYK.01.BHN, TA.EPYK.01.BHE
writing as xml data to file= orphanxml/EPYK.xml
saving to MongoDB site and channel collections
net:sta:loc= TA : EPYK :  for time span  2012-10-10T00:00:00.000000Z  to  2051-01-01T00:00:00.000000Z  added to site collection
net:sta:loc:chan= TA : EPYK :  : BHE for time span  2012-10-10T00:00:00.000000Z  to  2051-01-01T00:00:00.000000Z  added to channel collection
net:sta:loc:chan= TA : EPYK :  : BHN for time span  2012-10-10T00:00:00.000000Z  to  2051-01-01T00:00:00.000000Z  added to channel collection
net:st

net:sta:loc:chan= TA : TCOL : 01 : BHZ for time span  2012-10-28T02:49:00.000000Z  to  2012-11-01T22:14:59.000000Z  added to channel collection
net:sta:loc:chan= TA : TCOL : 01 : BHZ for time span  2012-11-01T22:15:00.000000Z  to  2013-06-14T00:59:59.000000Z  added to channel collection
net:sta:loc:chan= TA : TCOL : 01 : BHZ for time span  2013-06-14T01:00:00.000000Z  to  2017-09-14T01:29:59.000000Z  added to channel collection
net:sta:loc:chan= TA : TCOL : 01 : BHE for time span  2012-10-08T00:00:00.000000Z  to  2012-10-28T02:48:59.000000Z  already in channel collection - ignored
net:sta:loc:chan= TA : TCOL : 01 : BHE for time span  2012-10-28T02:49:00.000000Z  to  2012-11-01T22:14:59.000000Z  already in channel collection - ignored
net:sta:loc:chan= TA : TCOL : 01 : BHE for time span  2012-11-01T22:15:00.000000Z  to  2013-06-14T00:59:59.000000Z  already in channel collection - ignored
net:sta:loc:chan= TA : TCOL : 01 : BHN for time span  2012-10-08T00:00:00.000000Z  to  2012-10-28T02

No B channel data for station= WLVO  for any net code
Ignoring this station
Attempting to download xml data for station= LSQQ
No B channel data for station= LSQQ  for any net code
Ignoring this station
Attempting to download xml data for station= BMRO
No B channel data for station= BMRO  for any net code
Ignoring this station
Attempting to download xml data for station= BRCO
No B channel data for station= BRCO  for any net code
Ignoring this station
Attempting to download xml data for station= ALFO
No B channel data for station= ALFO  for any net code
Ignoring this station
Attempting to download xml data for station= DRCO
No B channel data for station= DRCO  for any net code
Ignoring this station
Attempting to download xml data for station= PLVO
No B channel data for station= PLVO  for any net code
Ignoring this station
Attempting to download xml data for station= MEDO
No B channel data for station= MEDO  for any net code
Ignoring this station
Attempting to download xml data for statio

The verbose output of that script above shows the problem sites are all created by new sites coming into the TA data stream from testing in Alaska.  Other seem extraneous stations that do not have broadband channels.  

Note that should eventually be removed before release:  There is a bug in the save_inventory script somewhere from some initialization related to the obnoxious behavior of insert_one. insert_one adds the object_id of the inserted document to the dict it is passed as an argument.  That creates a duplicate key condition if that id is not cleared.   Couldn't find it without a more careful debugging but found I could get through the whole processing by just hitting run again and again until it finished.  Apparently the initialization is on a second pass after a file is processed.

Let's verify what stations still have null net codes

In [14]:
from mspasspy.preprocessing.css30.dbarrival import find_null_net_stations
ret=find_null_net_stations(db)
print(ret)

{'ELFO', 'MATQ', 'TYNO', 'LATQ', 'ACTO', 'CHGQ', 'CLWO', 'POKR', 'BASO', 'BWLO', 'BELQ', 'Y22E', 'EPYK', 'PKRO', 'DELO', 'STCO', 'ORHO', 'BUKO', 'TOBO', 'TORO', 'DRWO', 'TCOL', 'BANO', 'TOLK', 'HDA', 'WLVO', 'LSQQ', 'BMRO', 'BRCO', 'ALFO', 'DRCO', 'PLVO', 'MEDO', 'KLBO', 'ORIO', 'PLIO', 'ALGO', 'PEMO'}


The output from above suggests all these data are not of interest for what we are assembling (Alaska test sites and sites that don't have broadband sensors) but we should do what we can.   The next block test the set returned above and reruns the function to try to set the net code from the site table:

In [15]:
for sta in ret:
    n=set_netcode_time_interval(db,sta,net='TA')
    print('set net code to TA for ',n,' arrival documents with picks for station=',sta)


set net code to TA for  270  arrival documents with picks for station= ELFO
set net code to TA for  287  arrival documents with picks for station= MATQ
set net code to TA for  276  arrival documents with picks for station= TYNO
set net code to TA for  184  arrival documents with picks for station= LATQ
set net code to TA for  288  arrival documents with picks for station= ACTO
set net code to TA for  234  arrival documents with picks for station= CHGQ
set net code to TA for  260  arrival documents with picks for station= CLWO
set net code to TA for  270  arrival documents with picks for station= POKR
set net code to TA for  302  arrival documents with picks for station= BASO
set net code to TA for  260  arrival documents with picks for station= BWLO
set net code to TA for  149  arrival documents with picks for station= BELQ
set net code to TA for  270  arrival documents with picks for station= Y22E
set net code to TA for  287  arrival documents with picks for station= EPYK
set net code

Let's finish this by verifying we no longer have any null net entries in the arrival collection:

In [16]:
from mspasspy.preprocessing.css30.dbarrival import find_null_net_stations
ret=find_null_net_stations(db)
print(ret)

set()


## 5. Indexing Waveform Data
For this tutorial we will be reading a set of miniseed files derived from the files downloaded by web services with obspy as described in section 3 above.   As noted earlier I elected to concatenate the large number of miniseed files obspy's downloader wrote to create files that would work more efficiently on HPC systems.  For readers familiar with seismic reflection processing the files we will be working with in this and the next section can be thought of as a bundle that defines a common source (shot) gather.   In MsPASS we abstract any group of seismic data objects that have some relationship into a container we call and Ensemble object.   For obspy users an ensemble is similar in concept to the obspy Stream object but more generic.  

For the preprocessing phase the next step we need to accomplish is building an index that defines the contents of the data we are aiming to assemble.   We will again be using MongoDB to accomplish this task.  We will create a set of documents that describe our data in it's raw form.   In the following sections we will edit these documents to add additional metadata needed to complete the data set.  Then we will run an example MsPASS run with spark that will be used to preprocessind and time window these data before writing them into the processing collection with call *wf*.   

The next code block runs a small function we can use to build our index.   Run the next code block and we will discuss what it does in afterwards by mixing in some words with database queries to explain the concepts.

In [17]:
from mspasspy.preprocessing.seed.ensembles import (dbsave_seed_ensemble_file)
for i in range(10):
    fname="wf/2012/event{0}.mseed".format(i+1)
    print('Building index for miniseed file=',fname)
    dbsave_seed_ensemble_file(db,fname)

Building index for miniseed file= wf/2012/event1.mseed
Building index for miniseed file= wf/2012/event2.mseed
Building index for miniseed file= wf/2012/event3.mseed
Building index for miniseed file= wf/2012/event4.mseed
Building index for miniseed file= wf/2012/event5.mseed


/home/pavlis/.local/lib/python3.8/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 273015296 to 273015423.
  warnings.warn(_w, InternalMSEEDWarning)
/home/pavlis/.local/lib/python3.8/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 273015424 to 273015551.
  warnings.warn(_w, InternalMSEEDWarning)
/home/pavlis/.local/lib/python3.8/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 273015552 to 273015679.
  warnings.warn(_w, InternalMSEEDWarning)
/home/pavlis/.local/lib/python3.8/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 273015680 to 273015807.
  warnings.warn(_w, InternalMSEEDWarning)
/home/pavlis/.local/lib/python3.8/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: readMSEEDBuffer

something threw an exception - this needs detailed handlers
Building index for miniseed file= wf/2012/event6.mseed
Building index for miniseed file= wf/2012/event7.mseed
Building index for miniseed file= wf/2012/event8.mseed
Building index for miniseed file= wf/2012/event9.mseed
Building index for miniseed file= wf/2012/event10.mseed


Notice that event5 had read errors.  That problem, unfortunately, is too common with waveform data downloaded by web services.  This function makes no attempt to salvage that data so it is lost until we attempt further detective work.   We will ignore that problem for now, but emphasize to the reader that is part of the tedious work that is required today to assemble the most complete dataset.  What complete means is dependent on the project so we treat solving that particular problem as an exercise for the student.  We intentionally leave the data slightly dirty to emphasize this point.

The following script shows only 9 of the 10 files were successfully processed and prints the contents of the first document in a prettier for (the full collection has 9 similar documents) 

In [18]:
dbwf=db.seed_data.ensemble
n=dbwf.count_documents({})
print('seedwf collection has ',n,' documents - should have been 10')
doc=dbwf.find_one()
print('Ensemble Metadata')
for k in doc:
    if k!='members':
        print(k,doc[k])
print('Ensemble Member Metadata')
n=0
for d in doc['members']:
    print('Metadata dict for member number ',n)
    print(d)
    n+=1
    

seedwf collection has  9  documents - should have been 10
Ensemble Metadata
_id 5fb2f2311d06656b8f8a95cb
dir /home/pavlis/testdata/tutorial/wf/2012
dfile event1.mseed
format mseed
member_type TimeSeries
mover obspy_seed_ensemble_reader
starttime 1356943580.750001
endtime 1356947180.744538
Ensemble Member Metadata
Metadata dict for member number  0
{'net': '2G', 'sta': 'BHE', 'chan': '', 'starttime': 1356943580.799998, 'endtime': 1356947180.749998, 'sampling_rate': 20.0, 'delta': 0.05, 'npts': 72000, 'calib': 1.0}
Metadata dict for member number  1
{'net': '7D', 'sta': 'BHZ', 'chan': '', 'starttime': 1356943580.7527978, 'endtime': 1356947180.7327979, 'sampling_rate': 50.0, 'delta': 0.02, 'npts': 180000, 'calib': 1.0}
Metadata dict for member number  2
{'net': '7D', 'sta': 'BH2', 'chan': '', 'starttime': 1356943580.7556999, 'endtime': 1356947180.7357, 'sampling_rate': 50.0, 'delta': 0.02, 'npts': 180000, 'calib': 1.0}
Metadata dict for member number  3
{'net': '2G', 'sta': 'BHN', 'chan':

{'net': 'TA', 'sta': 'BHN', 'chan': '', 'starttime': 1356943580.7500002, 'endtime': 1356947180.7500002, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144001, 'calib': 1.0}
Metadata dict for member number  1403
{'net': 'TA', 'sta': 'BHZ', 'chan': '', 'starttime': 1356943580.7500002, 'endtime': 1356947180.7500002, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144001, 'calib': 1.0}
Metadata dict for member number  1404
{'net': 'TA', 'sta': 'BHN', 'chan': '', 'starttime': 1356943580.774998, 'endtime': 1356947180.749998, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144000, 'calib': 1.0}
Metadata dict for member number  1405
{'net': 'TA', 'sta': 'BHE', 'chan': '', 'starttime': 1356943580.7500002, 'endtime': 1356947180.725, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144000, 'calib': 1.0}
Metadata dict for member number  1406
{'net': 'TA', 'sta': 'BHE', 'chan': '', 'starttime': 1356943580.7500002, 'endtime': 1356947180.7500002, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144001, 'ca

Metadata dict for member number  3069
{'net': 'XI', 'sta': 'BHZ', 'chan': '', 'starttime': 1356943580.77, 'endtime': 1356947180.745, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144000, 'calib': 1.0}
Metadata dict for member number  3070
{'net': 'XO', 'sta': 'BHN', 'chan': '', 'starttime': 1356943580.755, 'endtime': 1356947180.7299998, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144000, 'calib': 1.0}
Metadata dict for member number  3071
{'net': 'XO', 'sta': 'BHN', 'chan': '', 'starttime': 1356943580.7649999, 'endtime': 1356947180.74, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144000, 'calib': 1.0}
Metadata dict for member number  3072
{'net': 'XO', 'sta': 'BHZ', 'chan': '', 'starttime': 1356943580.76, 'endtime': 1356947180.7350001, 'sampling_rate': 40.0, 'delta': 0.025, 'npts': 144000, 'calib': 1.0}
Metadata dict for member number  3073
{'net': 'XO', 'sta': 'BHN', 'chan': '', 'starttime': 1356943580.7649999, 'endtime': 1356947180.74, 'sampling_rate': 40.0, 'delta': 0.025, 'np

Here are some key points about this rather large output:
- We write the documents that define this index in a collection called "seed_data.ensemble".   Note the "." character is treated as part of the name and is not significant.  i.e. the collection name is literally "seed_data.ensemble".  It is, however, common practice to use names like this in MongoDB to group collections having some kind of conceptual relationship.  In this case, it is one of several possible collections that could be created from seed data.  Other raw data types are expected to appear as collections with similar names (e.g. segy_data, ph5_data, etc.) 
- Readers familiar with CSS3.0 and/or Antelope may find it useful think of seed_data.ensemble as comparable to the css3.0 table called wfdisc.   The data are not loaded into MongoDB's work area.  seed_data.ensemble documents are only an index defining data holdings in files.  The approach here is strictly file based, but could be completely different.  e.g. instead of a file with properties defined by dir, dfile, and format we could define a web services URL.    
- Under the hood here we use obspy's miniseed reader.
- This collection uses what MongoDB calls subdocuments.  There is one subdocument for each channel of data in this event gather.  The subdocuments are a list of dict containers derived from the obspy stats array for each Trace object obspy reader creates in cracking the miniseed file.  The list is indexed by the key "members".  
- Each file also generates a set of metadata we tag in the print script above as "Ensemble Metadata".  We do that because the will literally be used for that purpose in the MsPASS Ensemble container later, but more importantly because it fits a generic concept.  That is, an "Ensemble" in MsPASS means a group of atomic objects that have some generic relationship.  In this case, the relationship is there are all connected to a single earthquake. The next thing we will need to do is link the ensemble to the source with with it is associated.  We will do that with pure database manipulations of small tables for efficiency. 

## 6. Source Association
The general problem of associating a set of waveforms to a metadata describing the source of the transient signals to be analyzed can be a thorny one.  For this tutorial we can use a very simple algorithm, however, due to the specific way I downloaded these data with obspy.   That is, all the event files were referenced to a be similar to seismic reflection shot gathers such that the first data sample for all channels is approximately the estimated origin time of the earthquake.  You can see that in the dump above because all the channels have a start time within a few 10s of ms of epoch time 1356943580.7.  Hence, the algorithm we will run below just looks for events in the source collection for which the origin time is within a tolerance of the average start time of the ensemble members (stored in the ensemble metadata with the key starttime).  

In [19]:
from mspasspy.preprocessing.seed.ensembles import link_source_collection
link_source_collection(db)
    

debug - query: {'time': {'$gte': 1356943570.750001, '$lte': 1356943590.750001}}
range between  2012-12-31T08:46:10.750001Z -> 2012-12-31T08:46:30.750001Z
debug - found  1  documents
debug - matchid and source_id= 5fb2f2311d06656b8f8a95cb 5fb2ea046fd117975915bc05
debug - query: {'time': {'$gte': 1356918291.5, '$lte': 1356918311.5}}
range between  2012-12-31T01:44:51.500000Z -> 2012-12-31T01:45:11.500000Z
debug - found  1  documents
debug - matchid and source_id= 5fb2f2341d06656b8f8a95cc 5fb2ea046fd117975915bc06
debug - query: {'time': {'$gte': 1356900558.550001, '$lte': 1356900578.550001}}
range between  2012-12-30T20:49:18.550001Z -> 2012-12-30T20:49:38.550001Z
debug - found  1  documents
debug - matchid and source_id= 5fb2f2381d06656b8f8a95cd 5fb2ea046fd117975915bc07
debug - query: {'time': {'$gte': 1356870071.775, '$lte': 1356870091.775}}
range between  2012-12-30T12:21:11.775000Z -> 2012-12-30T12:21:31.775000Z
debug - found  1  documents
debug - matchid and source_id= 5fb2f23c1d0665

Now that little function associated source data downloaded with web services by obspy.   Those events are not from the same source as the events ANF used to when they picked P waves from the TA data stream.   Further, a large fraction of the station in our download did not come on the real time data stream to ANF so they would not have picks in the css3.0 arrival data.  Recall our objective in this tutorial is to extract only waveforms that have a pick made by ANF.   

Our first step to do that is we need to also load the source locations in the ascii file we used to generate the arrival collection into our source collection.  We first run this small function to get the unique sources found in the (large) ascii table we created earlier.   Run the next cell and we will view the output below:

In [20]:
from mspasspy.preprocessing.css30.dbarrival import extract_unique_css30_sources
events=extract_unique_css30_sources('catalog_data/usarray_tele2012.txt')
print('Found ',len(events),' unique sources from 2012 css3.0 data file')
# print just the first 10 events
n=0
for k in events:
    print('evid=',k,' contents:  ',events[k])
    n+=1
    if(n>=10):
        break

Found  906  unique sources from 2012 css3.0 data file
evid= 206397  contents:   {'evid': 206397, 'latitude': 12.011, 'longitude': 143.505, 'depth': 10.0, 'time': 1325377805.03}
evid= 206398  contents:   {'evid': 206398, 'latitude': -11.372, 'longitude': 166.224, 'depth': 66.7, 'time': 1325379008.01}
evid= 206401  contents:   {'evid': 206401, 'latitude': 31.416, 'longitude': 138.155, 'depth': 348.5, 'time': 1325395674.5}
evid= 206408  contents:   {'evid': 206408, 'latitude': 12.018, 'longitude': 143.607, 'depth': 6.5, 'time': 1325432652.07}
evid= 206439  contents:   {'evid': 206439, 'latitude': -14.748, 'longitude': 167.44, 'depth': 221.6, 'time': 1325652454.05}
evid= 206454  contents:   {'evid': 206454, 'latitude': -10.67, 'longitude': 166.375, 'depth': 75.9, 'time': 1325703417.29}
evid= 206458  contents:   {'evid': 206458, 'latitude': -45.97, 'longitude': -76.014, 'depth': 10.0, 'time': 1325724873.21}
evid= 206459  contents:   {'evid': 206459, 'latitude': -17.691, 'longitude': -173.54

As the name of that function implies what it did was scan that large text file keeping all locations associated with unique evid values.  That is a potentially dangerous test if used on a poorly constructed css3.0 database, but the ANF databases are clean so we don't have an issue.  I state that because if anyone tries to extend this tutorial don't do so without making sure the evid values are unique with and between any file(s) you try to process this way.

We now use the function below to load this set of locations with the evid attributes into the source collection.  Note, however, that the number of attributes we store with these sources is much smaller than those loaded form the quakeml files.

In [21]:
from mspasspy.preprocessing.css30.dbarrival import load_css30_sources
n=load_css30_sources(db,events)
print('loaded ',n,' new documents into source collection')

loaded  906  new documents into source collection


The set of events we just loaded can be distinguished from those created from the quakeml file by the existence of the attribute "evid" in the documents we just saved;  the quakeml generated documents do not contain an evid attribute.  This short script demonstrates that using some low level MongoDB queries:

In [22]:
dbsource=db.source
n=dbsource.count_documents({})
print('Source collection currently has ',n,' total source documents')
query={'evid': {'$exists' : True}}
n=dbsource.count_documents(query)
print(n,' of the documents have evid set')

Source collection currently has  1867  total source documents
906  of the documents have evid set


Because evid values appear in arrival documents too, we will exploit that relation to associate arrivals with waveforms.  First, because evids are potentially dangerous we will run the following small function to post the unique id source_id to all arrival documents.  

In [23]:
from mspasspy.preprocessing.css30.dbarrival import set_source_id_from_evid
result=set_source_id_from_evid(db)
print('Number of arrival documents processed=',result[0])
print('Number of arrival documents updated=',result[1])
print('Number arrivals in set that did not match keyed by evid =',result[3])

Number of arrival documents processed= 353482
Number of arrival documents updated= 353482
Number arrivals in set that did not match keyed by evid = {}


# 7. Defining Indexes
A MongoDB database is heavily dependent upon a well defined index to improve preformance on large collections.  In this tutorial the only collection that will create problems for us is the arrival collection.   We saw that earlier when we updated every document in the collection. Indices are best built on static collections as they slow writes.  We didn't do this step earlier because we were building the arrival collection and doing as many writes as reads. For the next step we will only be reading form arrival (a lot) so we need to build an index.  Experience shows this will speed processing by 1 to 3 orders of magnitude.  

We are going to build the index with these three keys to support the queries we will be running on arrival below:   source_id, net, and sta.  The MongoDB syntax to build an index is a bit weird but this serves as a good example:

In [24]:
db.arrival.create_index(
[
    ('source_id',1),
    ('net', 1),
    ('sta',1)
])

'source_id_1_net_1_sta_1'

Noting two things about this:
1.  The pymongo api uses a different data structure from the mongo shell.  This warning is important if you do an internet search for building an index in MongoDB - build on pymongo examples as mongo shell commands require a translation.
2.  The magic 1 is a cryptic shorthand for the more verbose "ASCENDING" tag you will find in some sources.  You can also use "DESCENDING" or -1 to reverse the search order for the index.   Reversing the order of the index search has no merit here, but you can imagine examples where it would matter.

# 8.  Ensemble Processing
## 8.1. Cross-referencing source data
We previously wrote an index for common source gather files we downloaded in a collection we called seed_data.ensembles.  The following function takes the time range of that data and associates a document in the source collection with the file writing the source_id to the ensemble's index document.  We will need that in the reduction workflow that concludes this tutorial to link arrivals to the waveform data.

This function also has to deal with another not so small detail.  We have duplicate source locations for most of the events in the source collection.  The reason is we loaded the catalog obspy downloaded and the one ANF used in measuring the picks in the arrival collection.  We want to prefer the ones form ANF as they are the ones we can easily link to arrival.

In [26]:
from mspasspy.preprocessing.seed.ensembles import link_source_collection
# this function is like subroutine and returns nothhing
link_source_collection(db,prefer_evid=True)

debug - query: {'time': {'$gte': 1356943570.750001, '$lte': 1356943590.750001}}
range between  2012-12-31T08:46:10.750001Z -> 2012-12-31T08:46:30.750001Z
debug - found  1  documents
debug - matchid and source_id= 5fb2f2311d06656b8f8a95cb 5fb2ea046fd117975915bc05
link_source_collection(WARNING): unique match for source at time= 2012-12-31T08:46:20.750001Z  does not have evid set but function was called with prefer_evid true
debug - query: {'time': {'$gte': 1356918291.5, '$lte': 1356918311.5}}
range between  2012-12-31T01:44:51.500000Z -> 2012-12-31T01:45:11.500000Z
debug - found  1  documents
debug - matchid and source_id= 5fb2f2341d06656b8f8a95cc 5fb2ea046fd117975915bc06
link_source_collection(WARNING): unique match for source at time= 2012-12-31T01:45:01.500000Z  does not have evid set but function was called with prefer_evid true
debug - query: {'time': {'$gte': 1356900558.550001, '$lte': 1356900578.550001}}
range between  2012-12-30T20:49:18.550001Z -> 2012-12-30T20:49:38.550001Z
de